In [1]:
import cv2
import numpy as np
import math
import scipy.fftpack

In [7]:
img = cv2.imread("kodim14.png")

ycbcr = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

In [8]:
def encode(ycbcr):
    y, cb, cr = cv2.split(ycbcr)
    dim = (int(ycbcr.shape[1]/2), int(ycbcr.shape[0]/2))
    
    resizedCb = cv2.resize(cb, dim)
    resizedCr = cv2.resize(cr, dim)
    

    return y, resizedCb, resizedCr

def decode(y, halfcb, halfcr):
    dim = (int(halfcb.shape[1]*2), int(halfcb.shape[0]*2))
    
    halfcb = np.array(halfcb, dtype='uint8')
    halfcr = np.array(halfcr, dtype='uint8')
    y = np.array(y, dtype='uint8')
    
    originalCb = cv2.resize(halfcb, dim)
    originalCr = cv2.resize(halfcr, dim)

    merged = cv2.merge((y, originalCb, originalCr))
    
    merged = cv2.cvtColor(merged, cv2.COLOR_YCrCb2BGR)
    
    cv2.imwrite('01.png',merged)
    return merged


In [9]:
def divide8(y, cb, cr):
    h, w = y.shape
        
    YBlocks = []
    CbBlocks = []
    CrBlocks = []
    
    
    for row in range(0, h, 8):
        for col in range(0, w, 8):
            YBlocks.append(y[row : row + 8, col : col + 8])
    
    h, w = cb.shape
            
    for row in range(0, h, 8):
        for col in range(0, w, 8):
            CbBlocks.append(cb[row : row + 8, col : col + 8])
            
    for row in range(0, h, 8):
        for col in range(0, w, 8):
            CrBlocks.append(cr[row : row + 8, col : col + 8])
    
    return YBlocks, CbBlocks, CrBlocks

def undivide8(YBlocks, CbBlocks, CrBlocks, w, h):
    
    for k in range(len(YBlocks)):
        i = k // (w / 8) # quel bloc (ligne)
        j = k % (w / 8) # quel bloc (colonne)
        block = YBlocks[k] 

        for n in range(8): # quel pixel dans le bloc (ligne)
            for m in range(8): #quel pixel dans le bloc (colonne
                y[int(i * 8 + n)][int(j * 8 + m)] = block[n][m]
                
    for k in range(len(CbBlocks)):
        i = k // (w / 16)
        j = k % (w / 16)
        block = CbBlocks[k]
        for n in range(8):
            for m in range(8):
                cb[int(i * 8 + n)][int(j * 8 + m)] = block[n][m]
                
    for k in range(len(CrBlocks)):
        i = k // (w / 16)
        j = k % (w / 16)
        block = CrBlocks[k]
        for n in range(8):
            for m in range(8):
                cr[int(i * 8 + n)][int(j * 8 + m)] = block[n][m]
                
    return y, cb, cr

In [16]:
def dctTransform(matrix):
    m = 8
    n = 8
    dct = [[0 for x in range(m)] for y in range(n)] 

    
    for ligne in range(m):
        for col in range(n):
            if (ligne == 0):
                ci = 1 / math.sqrt(m)
            else:
                ci = math.sqrt(2) / math.sqrt(m)
            if (col == 0):
                cj = 1 / math.sqrt(n)
            else:
                cj = math.sqrt(2) / math.sqrt(n)
            
            sum = 0
            for k in range(m):
                for l in range (n):
                    dct1 = matrix[k][l] * math.cos((2 * k + 1) * ligne * math.pi / (2 * m)) * math.cos((2 * l + 1) * col * math.pi / (2 * n))
                    sum = sum + dct1
            
            dct[ligne][col] = ci * cj * sum
            
    return dct

def reverseDctTransform(matrix):
    pass

def dct2(a):
    #https://inst.eecs.berkeley.edu/~ee123/sp16/Sections/JPEG_DCT_Demo.html
    return scipy.fftpack.dct( scipy.fftpack.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    #https://inst.eecs.berkeley.edu/~ee123/sp16/Sections/JPEG_DCT_Demo.html
    return scipy.fftpack.idct( scipy.fftpack.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

def dct(YBlocks, CbBlocks, CrBlocks):
    YBlocks = dct2(YBlocks)
    CbBlocks = dct2(CbBlocks)
    CrBlocks = dct2(CrBlocks)
    return YBlocks, CbBlocks, CrBlocks

def idct(YBlocks, CbBlocks, CrBlocks):
    YBlocks = idct2(YBlocks)
    CbBlocks = idct2(CbBlocks)
    CrBlocks = idct2(CrBlocks)
    return YBlocks, CbBlocks, CrBlocks
    

quantizationBlock =[  [16,  11,  10,  16,  24,  40,  51,  61],
  [12,  12,  14,  19,  26,  58,  60,  55],
  [14,  13,  16,  24,  40,  57,  69,  56],
  [14,  17,  22,  29,  51,  87,  80,  62],
  [18,  22,  37,  56,  68, 109, 103,  77],
  [24,  35,  55,  64,  81, 104, 113,  92],
  [49,  64,  78,  87, 103, 121, 120, 101],
  [72,  92,  95,  98, 112, 100, 103,  99]]

def quantizeBlocks(blocks):
    for k in range(len(blocks)):
        block = blocks[k]
        for i in range(8):
            for j in range(8):
                block[i][j] = round(block[i][j] / quantizationBlock[i][j]);
    return blocks

def dequantizeBlocks(blocks):
    for k in range(len(blocks)):
        block = blocks[k]
        for i in range(8):
            for j in range(8):
                block[i][j] = block[i][j] * quantizationBlock[i][j];
    return blocks


In [17]:
y, cb, cr = encode(ycbcr)

YBlocks, CbBlocks, CrBlocks = divide8(y, cb, cr)

YBlocks, CbBlocks, CrBlocks = dct(YBlocks, CbBlocks, CrBlocks)

YBlocks = quantizeBlocks(YBlocks)
CbBlocks = quantizeBlocks(CbBlocks)
CrBlocks = quantizeBlocks(CrBlocks)

# encode huffman



# DECOMPRESSION #


# decode huffman


YBlocks = dequantizeBlocks(YBlocks)
CbBlocks = dequantizeBlocks(CbBlocks)
CrBlocks = dequantizeBlocks(CrBlocks)


YBlocks, CbBlocks, CrBlocks = idct(YBlocks, CbBlocks, CrBlocks)

y, cb, cr = undivide8(YBlocks, CbBlocks, CrBlocks, y.shape[1], y.shape[0])


image = decode(y, cb, cr)